## Get our workspace ready

In [ ]:
# importing necessary tools
import datetime
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D
from math import ceil

import tensorflow as tf
print("TF version: ", tf.__version__)

In [ ]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("No")
else:
    print(device_name)
# print(tf.test.is_gpu_available())

## Getting our data ready (turning into tensors)

In [ ]:
train_df = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Train.csv')
train_df.describe()

In [ ]:
labelNames = ['20 km/h', '30 km/h', '50 km/h', '60 km/h', '70 km/h', '80 km/h', '80 km/h end', '100 km/h', '120 km/h', 'No overtaking',
               'No overtaking for tracks', 'Crossroad with secondary way', 'Main road', 'Give way', 'Stop', 'Road up', 'Road up for track', 'Brock',
               'Other dangerous', 'Turn left', 'Turn right', 'Winding road', 'Hollow road', 'Slippery road', 'Narrowing road', 'Roadwork', 'Traffic light',
               'Pedestrian', 'Children', 'Bike', 'Snow', 'Deer', 'End of the limits', 'Only right', 'Only left', 'Only straight', 'Only straight and right', 
               'Only straight and left', 'Take right', 'Take left', 'Circle crossroad', 'End of overtaking limit', 'End of overtaking limit for track']
train_df["ClassName"] = [labelNames[Id] for Id in train_df["ClassId"]]
train_df = train_df.drop(['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2'], axis = 1)
train_df.head()

In [ ]:
train_df['ClassName'].value_counts().plot.bar(figsize=(20, 10))
train_df['ClassName'].value_counts().median()

## Getting images and their labels

In [ ]:
# Create pathnames from image Id's
filenames = ['/kaggle/input/gtsrb-german-traffic-sign/' + fname for fname in train_df['Path']]
filenames[:10]

In [ ]:
labels = train_df['ClassId'].to_numpy()
labels

In [ ]:
unique_signs = np.unique(labels)
len(unique_signs)

In [ ]:
# Converting the labels into one hot encoding
labels = tf.keras.utils.to_categorical(labels, 43)
labels[0]

In [ ]:
len(labels)

## Creating Validation set

In [ ]:
# Create X & y variables
X = filenames
y = labels

# Splitting our data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)
len(X_train), len(y_train), len(X_val), len(y_val)

## Processing image and turning into Tensors

### Turning data into batches

In [ ]:
IMG_SIZE = 32


def process_image(image_path, label=None, denoise=''):
    """
    Takes an image file path and turns the image into a Tensor.
    """
    # Read in an image file
    image = tf.io.read_file(image_path)
    # Turn the jpeg image into numerical Tensor with 3 colour channels (Red, Green, Blue)
    image = tf.image.decode_png(image, channels=3)
    # Convert the colour channel values from 0-255 to 0-1 values
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Resize the image to our desired value (32, 32)
    image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
    #if denoise:
    #    image = denoise_image(denoise)
    return image

In [ ]:
# Create a simple function to return tuple
def get_image_label (image_path, label, denoise=''):
    """
    Takes an image file path name and the assosciated label,
    processes the image and reutrns a typle of (image, label).
    """
    image = process_image(image_path, label, denoise)
    return image, label

In [ ]:
# Define batch size
BATCH_SIZE = 64

# Create a function to turn data into batches
def create_data_batches (X, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False, denoise=''):
    """
    Creates batches of data out of image (X) and label (y) pairs.
    Shuffles the data if it's training data but doesn't shuffle if it's validation dat
    a.
    Also accepts test data as input (no labels).
    """
    # If the data is a test dataset, we probably don't have labels
    if test_data:
        print("Creating test data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y), tf.constant([denoise]*len(X))))
        data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    # If the data is a valid dataset, we don't need to shuffle it
    elif valid_data:
        print("Creating validation dataset batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
        # Create (image, label) tuples (this also turns the iamge path into a preprocessed image)
        data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    else:
        print("Creating training dataset batches...")
        # Turn filepaths and labels into Tensors
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
        # Shuffling pathnames and labels before mapping image processor function is faster than shuffling images
        data = data.shuffle(buffer_size=len(X))
        # Create (image, label) tuples (this also turns the iamge path into a preprocessed image) and turning into batches
        data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

In [ ]:
# Creating training and validation batches
train_data = create_data_batches(X_train, y_train, test_data=False)
val_data = create_data_batches(X_val, y_val, valid_data=True)

In [ ]:
# Check out the different attributes of our data batches
train_data.element_spec, val_data.element_spec

## Visualizing Data Batches

In [ ]:
# Create a function for viewing images in a data batch
def show_25_images(images, labels):
    """
    Displays a plot of 25 images and their labels from a data batch.
    """
    plt.figure(figsize=(15,15))
    for i in range(25):
        ax = plt.subplot(5, 5, i+1)
        plt.imshow(images[i])
        classId = unique_signs[labels[i].argmax()]
        plt.title(f"{classId}: {labelNames[classId]}")
        plt.axis("off")

In [ ]:
# Visualizing traing batch
train_images, train_labels = next(train_data.as_numpy_iterator())
show_25_images(train_images, train_labels)

In [ ]:
# Visualizing validation data
val_images, val_labels = next(val_data.as_numpy_iterator())
show_25_images(val_images, val_labels)

## Building the model

In [ ]:
# Setup input shape to the model
INPUT_SHAPE = [IMG_SIZE, IMG_SIZE, 3]

# Setup the output shape
OUTPUT_SHAPE = len(unique_signs)

In [ ]:
# Creating CNN Model
def traffic_sign_net(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(43, activation='softmax'))
    return model

from tensorflow import keras

def vggnet(input_shape, output_shape):
    inp = keras.layers.Input(shape=input_shape, name='image_input')

    vgg_model = keras.applications.vgg16.VGG16(include_top=False, input_shape=input_shape)
    vgg_model.trainable = True
    
    x = vgg_model(inp)
    x = keras.layers.Flatten(name='flatten')(x)
    x = keras.layers.Dense(512, activation='relu', name='fc1')(x)
    x = keras.layers.Dense(512, activation='relu', name='fc2')(x)
    x = keras.layers.Dense(43, activation='softmax', name='predictions')(x)
    new_model = keras.models.Model(inputs=inp, outputs=x)
    new_model.compile(optimizer='adam', loss='categorical_crossentropy', 
                   metrics=['accuracy'])
    return new_model

In [ ]:
# Create a function that creates model
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, vgg=False):
    # Setup the model layers
    if vgg:
        model = vggnet(input_shape, output_shape)
    else:    
        model = traffic_sign_net(input_shape=input_shape)

    # Compile the model
    print("Compiling the model")
    model.compile(
        loss = tf.keras.losses.CategoricalCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(),
        metrics = ["accuracy"]
    )
    return model

In [ ]:
model = create_model()
model.summary()

## Creating Early Stopping callback

In [ ]:
# Create early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

## Training our model

In [ ]:
NUM_EPOCHS = 12

In [ ]:
# Build a fn to train and return a trained model
def train_model():
    """
    Trains a given model and returns the trained version.
    """
    # Create a model
    model = create_model()
    # Fit the model to the data passing it the callbacks we created
    model.fit(x=train_data,
        epochs=NUM_EPOCHS,
        validation_data=val_data,
        validation_freq=1,
        callbacks=[early_stopping]
             )
    return model


In [ ]:
TRAIN = False

#model_name = "traffic-sign-net"
model_name = "vggnet"

if TRAIN:
    with tf.device("/GPU:0"):
        model = train_model()
    model.save(model_name)
    TRAIN = False
else:
    from tensorflow import keras
    model = keras.models.load_model("/kaggle/input/trafficsignnet/traffic-sign-net")
    test_model = keras.models.load_model("/kaggle/input/vggnet/vggnet")

## Creating test dataset batches

In [ ]:
# Function to convert probabilities to labels
def get_pred_label(prediction_probabilities):
    """
    Turns an array of prediction probabilities into a label.
    """
    return unique_signs[np.argmax(prediction_probabilities)]

def get_predictions(X_test):
    predictions = model.predict(X_test, verbose=0)
    # Turning probabilities to labels
    y_pred = []
    for i in predictions:
        y_pred.append(get_pred_label(i))
    return y_pred

In [ ]:
def print_results(y_test, y_pred, epsilons=None, time=None):
    import seaborn as sn
    from sklearn import metrics
    
    test_accuracy = accuracy_score(y_test, y_pred)
    print ("test accuracy:", test_accuracy)
    y_true = y_test
    print ("Precision", metrics.precision_score(y_true, y_pred, average='macro'))
    print ("Recall", metrics.recall_score(y_true, y_pred, average='micro'))
    print ("f1_score", metrics.f1_score(y_true, y_pred, average='weighted'))
    print ("Confusion matrix:")

    confusion_matrix = metrics.confusion_matrix(y_true, y_pred)
    normalized_confusion_matrix = confusion_matrix / confusion_matrix.astype(np.float).sum(axis=1, keepdims=True)
    sn.heatmap(normalized_confusion_matrix, annot=False)
    plt.show()

In [ ]:
labelNames = ['20 km/h', '30 km/h', '50 km/h', '60 km/h', '70 km/h', '80 km/h', '80 km/h end', '100 km/h', '120 km/h', 'No overtaking',
               'No overtaking for tracks', 'Crossroad with secondary way', 'Main road', 'Give way', 'Stop', 'Road up', 'Road up for track', 'Brock',
               'Other dangerous', 'Turn left', 'Turn right', 'Winding road', 'Hollow road', 'Slippery road', 'Narrowing road', 'Roadwork', 'Traffic light',
               'Pedestrian', 'Children', 'Bike', 'Snow', 'Deer', 'End of the limits', 'Only right', 'Only left', 'Only straight', 'Only straight and right', 
               'Only straight and left', 'Take right', 'Take left', 'Circle crossroad', 'End of overtaking limit', 'End of overtaking limit for track']

test_df = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Test.csv')
test_df = test_df.drop(['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2'], axis=1)

test_df["ClassName"] = [labelNames[Id] for Id in test_df["ClassId"]]
test_df.head()

In [ ]:
test_df.describe()

In [ ]:
test_df['ClassName'].value_counts().plot.bar(figsize=(20, 10))
test_df['ClassName'].value_counts().median()

In [ ]:
test_img_paths = ['/kaggle/input/gtsrb-german-traffic-sign/' + path for path in test_df['Path']]
test_img_paths[:10]

In [ ]:
y_test = list(test_df['ClassId'])
y_test_onehot = tf.keras.utils.to_categorical(y_test, 43)
X_test = create_data_batches(test_img_paths, y_test, test_data=True)
print(y_test[0])
print(y_test_onehot[0])

In [ ]:
# Visualizing test batch
test_images, test_labels = next(X_test.as_numpy_iterator())
show_25_images(test_images, y_test_onehot)

## Making and Evaluating predictions using a trained model on test data

In [ ]:
y_pred = get_predictions(X_test)
print_results(y_test, y_pred)

# Attacks

## foolbox
https://github.com/bethgelab/foolbox

In [ ]:
y_test = list(test_df['ClassId'])
y_test_onehot = tf.keras.utils.to_categorical(y_test, 43)
X_test = create_data_batches(test_img_paths, y_test, test_data=True)
print(y_test[0])
print(y_test_onehot[0])

test_images, test_labels = next(X_test.as_numpy_iterator())
plt.imshow(test_images[0])

In [ ]:
!pip install foolbox

In [ ]:
import time
import foolbox # to make it easier trying out different attacks
from foolbox import TensorFlowModel, accuracy, samples, Model
import eagerpy as ep

In [ ]:
# pre = dict(flip_axis=-1, mean=[104.0, 116.0, 123.0])  # RGB to BGR
fmodel: Model = TensorFlowModel(model, bounds=(0, 255), preprocessing=None)
#fmodel = fmodel.transform_bounds((0, 1))

In [ ]:
images = ep.from_numpy(fmodel.dummy, np.stack(test_images)).raw
labels = ep.from_numpy(fmodel.dummy, np.stack(test_labels)).raw

images, labels = ep.astensors(images, labels)

In [ ]:
from tqdm import tqdm

def show_side_by_side(img_list, epsilons, index=0):
    n = len(img_list)
    plt.figure(figsize=(15,6))
    for i in range(n):
        img = img_list[i][index].numpy().astype("float32")
        ax = plt.subplot(1, n, i+1)
        if epsilons[i]:
            title = f"$\epsilon$ = {epsilons[i]}"
        else:
            title = "original"
        ax.set_title(title)
        plt.axis("off")
        plt.imshow(img)
    plt.show()

# calculate and report the robust accuracy (the accuracy of the model when it is attacked)
def print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed, index=0):
    if False: #(raw_advs[-1] - clipped_advs[-1]).numpy().any():
        print("INFO: There is a difference between 'raw_advs' and 'clipped_advs'")
    print(f"total time (for {len(success)} iterations): {t_elapsed:.2f} seconds.")
    robust_accuracy = 1 - success.numpy().mean(axis=-1)
    print("robust accuracy for perturbations with")
    for eps, acc in zip(epsilons, robust_accuracy):
        print(f"  eps = {eps:<5}: {acc.item() * 100:4.3f} %")
    show_side_by_side(raw_advs, epsilons, index=index)
    
def attack_and_predict(X_test, attack, epsilons): # not nice, but cannot get it to work any other way...
    preds_list = []
    total = int(len(y_test) / BATCH_SIZE)
    for i, (images, labels) in tqdm(enumerate(X_test), total=total):
        raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
        for j in range(len(epsilons)):
            if i == 0:
                preds_list.append([])
            preds = get_predictions(raw_advs[j])
            preds_list[j] += preds
        # print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed)
    return preds_list

## Applying different Foolbox attacks
refer to https://foolbox.readthedocs.io/en/stable/modules/attacks.html for attack names (it is fun!)

*Hint:* If you get an error you might need to initialize the attack class with some values (will hopefully be specified in the error message)

In [ ]:
# apply the attack
attack = foolbox.attacks.L2AdditiveGaussianNoiseAttack()
epsilons = [
    0,
    0.1,
    1,
    10,
    30,
    100.0,
]
t = time.time()
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
t_elapsed = time.time() - t

print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed)

In [ ]:
# apply the attack
attack = foolbox.attacks.L2BasicIterativeAttack()
epsilons = [
    0.0,
    0.1,
    1,
    3,
    10,
    100
]
t = time.time()
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
t_elapsed = time.time() - t

print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed)

In [ ]:
# apply the attack
attack = foolbox.attacks.FGSM()
epsilons = [
    0.0,
    0.001,
    0.01,
    0.05,
    0.1,
    1.0,
]
t = time.time()
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
t_elapsed = time.time() - t

print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed)

In [ ]:
# apply the attack
attack = foolbox.attacks.BoundaryAttack()
fb_criterion = foolbox.criteria.Misclassification(labels)
epsilons = [
    0,
    0.001,
    0.01,
    0.1,
    0.5,
    1
]
t = time.time()
raw_advs, clipped_advs, success = attack(fmodel, images, criterion=fb_criterion, epsilons=epsilons)
t_elapsed = time.time() - t

print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed)

**ACCURACIES ARE WRONG!**

In [ ]:
y_pred = get_predictions(raw_advs[5].raw)
print_results(y_test[0:64], y_pred)

In [ ]:
    plt.figure(figsize=(18, 6))
    for i in range(12):
        ax = plt.subplot(2, 6, i+1)
        if i < 6:
            plt.imshow(images[i].raw)
            classId = unique_signs[y_test[i]]
        else:
            plt.imshow(raw_advs[-1][i-6].raw) 
            classId = unique_signs[y_pred[i-6]]           
        plt.title(f"{labelNames[classId]}")
        plt.axis("off")

In [ ]:
# apply the attack
TARGET_CLASS = 40
attack = foolbox.attacks.BoundaryAttack()
fb_criterion = foolbox.criteria.TargetedMisclassification(target_classes=labels / labels * TARGET_CLASS)
epsilons = [
    0,
    0.001,
    0.01,
    0.1,
    0.5,
    1
]
t = time.time()
raw_advs, clipped_advs, success = attack(fmodel, images, criterion=fb_criterion, epsilons=epsilons)
t_elapsed = time.time() - t

print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed)

In [ ]:
y_pred = get_predictions(raw_advs[5].raw)
print_results(y_test[0:64], y_pred)

In [ ]:
    plt.figure(figsize=(18, 6))
    for i in range(12):
        ax = plt.subplot(2, 6, i+1)
        if i < 6:
            plt.imshow(images[i].raw)
            classId = unique_signs[y_test[i]]
        else:
            plt.imshow(raw_advs[-1][i-6].raw) 
            classId = unique_signs[y_pred[i-6]]           
        plt.title(f"{labelNames[classId]}")
        plt.axis("off")

## Full attack evaluation (might take a long time!)

In [ ]:
# apply the attack
attack = foolbox.attacks.L2AdditiveGaussianNoiseAttack()
epsilons = [
    0.0,
    1,
    10,
    20,
    50,
    100.0,
]
t = time.time()
preds_list = attack_and_predict(X_test, attack, epsilons)
t_elapsed = time.time() - t

print(f"total time: {t_elapsed} seconds.")
for i, y_pred in enumerate(preds_list):
    print(f"epsilon: {epsilons[i]}")
    print_results(y_test, y_pred)
    print("")

In [ ]:
# apply the attack
attack = foolbox.attacks.L2BasicIterativeAttack()
epsilons = [
    0.0,
    0.1,
    0.5,
    1,
    2,
    10,
]
t = time.time()
preds_list = attack_and_predict(X_test, attack, epsilons)
t_elapsed = time.time() - t

print(f"total time: {t_elapsed} seconds.")
for i, y_pred in enumerate(preds_list):
    print(f"epsilon: {epsilons[i]}")
    print_results(y_test, y_pred)
    print("")

In [ ]:
# apply the attack
attack = foolbox.attacks.FGSM()
epsilons = [
    0.0,
    0.001,
    0.01,
    0.1,
    0.2,
    1.0,
]
t = time.time()
preds_list = attack_and_predict(X_test, attack, epsilons)
t_elapsed = time.time() - t

print(f"total time: {t_elapsed} seconds.")
for i, y_pred in enumerate(preds_list):
    print(f"epsilon: {epsilons[i]}")
    print_results(y_test, y_pred)
    print("")

In [ ]:
counter = 0
for i in range(len(y_test)):
    if preds_list[4][i] != y_test[i]:
        #print(f"{counter:>4}: {preds_list[4][i]} vs. {y_test[i]}")
        counter += 1
counter

## Gray Box attacks

In [ ]:
fmodel: Model = TensorFlowModel(test_model, bounds=(0, 255), preprocessing=None)

In [ ]:
# apply the attack
attack = foolbox.attacks.L2AdditiveGaussianNoiseAttack()
epsilons = [
    0,
    0.1,
    1,
    10,
    30,
    100.0,
]
t = time.time()
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
t_elapsed = time.time() - t

print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed)

In [ ]:
# apply the attack
attack = foolbox.attacks.L2BasicIterativeAttack()
epsilons = [
    0.0,
    0.1,
    1,
    3,
    10,
    100
]
t = time.time()
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
t_elapsed = time.time() - t

print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed)

In [ ]:
# apply the attack
attack = foolbox.attacks.FGSM()
epsilons = [
    0.0,
    0.001,
    0.01,
    0.05,
    0.1,
    1.0,
]
t = time.time()
raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
t_elapsed = time.time() - t

print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed)

In [ ]:
# apply the attack
attack = foolbox.attacks.BoundaryAttack()
fb_criterion = foolbox.criteria.Misclassification(labels)
epsilons = [
    0,
    0.001,
    0.01,
    0.1,
    0.5,
    1
]
t = time.time()
raw_advs, clipped_advs, success = attack(fmodel, images, criterion=fb_criterion, epsilons=epsilons)
t_elapsed = time.time() - t

print_stats(raw_advs, clipped_advs, success, epsilons, t_elapsed)

In [ ]:
y_pred = get_predictions(raw_advs[5].raw)
print_results(y_test[0:64], y_pred)

In [ ]:
    plt.figure(figsize=(18, 6))
    for i in range(12):
        ax = plt.subplot(2, 6, i+1)
        if i < 6:
            plt.imshow(images[i].raw)
            classId = unique_signs[y_test[i]]
        else:
            plt.imshow(raw_advs[-1][i-6].raw) 
            classId = unique_signs[y_pred[i-6]]           
        plt.title(f"{labelNames[classId]}")
        plt.axis("off")

# Defenses
## Adversarial Training

In [ ]:
fmodel: Model = TensorFlowModel(model, bounds=(0, 255), preprocessing=None)

In [ ]:
!mkdir /kaggle/working/Train/
for i in range(43):
    !mkdir /kaggle/working/Train/{i}

In [ ]:
attack = foolbox.attacks.FGSM()
epsilons = [0.05]

counter = 0

X = filenames
y = labels

train_data = create_data_batches(X, train_df['ClassId'], valid_data=True)
new_filenames = ['/kaggle/working/' + fname for fname in train_df['Path']]

if True:
    for i, (images, labels) in tqdm(enumerate(train_data), total=len(new_filenames) / BATCH_SIZE):
        raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
        
        for img in raw_advs[0]:
            #print(f"{new_filenames[counter]}, {X[counter]}")
            img = img.numpy() / np.max(img.numpy())
            plt.imsave(new_filenames[counter], img, format='png')
            counter += 1 

In [ ]:
# Converting the labels into one hot encoding
labels = train_df['ClassId'].to_numpy()
labels = tf.keras.utils.to_categorical(labels, 43)
len(labels)

In [ ]:
# Create X & y variables
X = new_filenames
y = labels

# Splitting our data into train and validation sets
adv_X_train, adv_X_val, adv_y_train, adv_y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)
c_X_train = adv_X_train + X_train
c_X_val = adv_X_val + X_val
c_y_train = np.append(adv_y_train, y_train, axis=0)
c_y_val = np.append(adv_y_val, y_val, axis=0)

In [ ]:
# Creating training and validation batches
train_data = create_data_batches(c_X_train, c_y_train)
val_data = create_data_batches(c_X_val, c_y_val, valid_data=True)

In [ ]:
# Visualizing traing batch
train_images, train_labels = next(train_data.as_numpy_iterator())
show_25_images(train_images, train_labels)

In [ ]:
orig_model = model

model = create_model()
model.summary()

In [ ]:
NUM_EPOCHS = 20

In [ ]:
TRAIN = True

%cd /kaggle/working/

#model_name = "traffic-sign-net"
model_name = "adv-traffic-sign-net"

if TRAIN:
    with tf.device("/GPU:0"):
        model = train_model()
    model.save(model_name)
    TRAIN = False
else:
    from tensorflow import keras
    model = keras.models.load_model("/kaggle/input/trafficsignnet/adv_traffic-sign-net")
    test_model = keras.models.load_model("/kaggle/input/vggnet/vggnet")

In [ ]:
y_pred = get_predictions(X_test)
print_results(y_test, y_pred)

In [ ]:
# apply the attack
attack = foolbox.attacks.FGSM()
epsilons = [
    0.0,
    0.001,
    0.01,
    0.1,
    0.2,
    1.0,
]
t = time.time()
preds_list = attack_and_predict(X_test, attack, epsilons)
t_elapsed = time.time() - t

print(f"total time: {t_elapsed} seconds.")
for i, y_pred in enumerate(preds_list):
    print(f"epsilon: {epsilons[i]}")
    print_results(y_test, y_pred)
    print("")

## Noise Reduction

In [ ]:
# !/opt/conda/bin/python3.7 -m pip install --upgrade scikit-image

In [ ]:
import skimage
print(skimage.__version__)

In [ ]:
!mkdir /kaggle/working/Test/

In [ ]:
test_img_paths = ['/kaggle/input/gtsrb-german-traffic-sign/' + path for path in test_df['Path']]

y_test = list(test_df['ClassId'])
y_test_onehot = tf.keras.utils.to_categorical(y_test, 43)
X_test = create_data_batches(test_img_paths, y_test, test_data=True)

In [ ]:
attack = foolbox.attacks.FGSM()
epsilons = [0.05]

counter = 0

test_img_paths = ['/kaggle/input/gtsrb-german-traffic-sign/' + path for path in test_df['Path']]

y_test = list(test_df['ClassId'])
y_test_onehot = tf.keras.utils.to_categorical(y_test, 43)
X_test = create_data_batches(test_img_paths, y_test, test_data=True)

#train_data = create_data_batches(X, train_df['ClassId'], valid_data=True)
new_filenames = ['/kaggle/working/' + path for path in test_df['Path']]

if True:
    for i, (images, labels) in tqdm(enumerate(X_test), total=ceil(len(new_filenames) / BATCH_SIZE)):
        raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
        
        for img in raw_advs[0]:
            #print(f"{new_filenames[counter]}, {X[counter]}")
            img = img.numpy() / np.max(img.numpy())
            plt.imsave(new_filenames[counter], img, format='png')
            counter += 1 

In [ ]:
X_test = create_data_batches(new_filenames, y_test, test_data=True)

# Visualizing test batch
test_images, test_labels = next(X_test.as_numpy_iterator())
show_25_images(test_images, y_test_onehot)

In [ ]:
TRAIN = False

#model_name = "traffic-sign-net"
model_name = "vggnet"

if TRAIN:
    with tf.device("/GPU:0"):
        model = train_model()
    model.save(model_name)
    TRAIN = False
else:
    from tensorflow import keras
    model = keras.models.load_model("/kaggle/input/trafficsignnet/traffic-sign-net")
    test_model = keras.models.load_model("/kaggle/input/vggnet/vggnet")

In [ ]:
from skimage.restoration import denoise_bilateral, denoise_wavelet, denoise_nl_means, denoise_tv_chambolle
#from skimage.filters import median

def denoise_and_predict(X_test, y_test, denoise): # not nice, but cannot get it to work any other way...
    pred_list = []
    total = ceil(len(y_test) / BATCH_SIZE)
    for i, (images, labels) in tqdm(enumerate(X_test), total=total):
        img_list = []
        for img in images:
            img = img.numpy()
            # plt.imshow(img)
            # plt.show()
            if denoise == "bilateral":
                img_list.append(denoise_bilateral(img, multichannel=True))
            elif denoise == "nl_means":
                img_list.append(denoise_nl_means(img, multichannel=True))  
            elif denoise == "tv_chambolle":
                img_list.append(denoise_wavelet(img, multichannel=True))           
            else: 
                img_list.append(img)
            # plt.imshow(img_list[-1])
            # plt.show()
        batch = tf.constant(img_list)
        preds = get_predictions(batch)
        pred_list += preds
        #print_stats([batch], [batch], success, epsilons, t_elapsed)
    return pred_list

In [ ]:
def show_images(img_list, titles, index=0):
    n = len(img_list)
    plt.figure(figsize=(15,6))
    for i in range(n):
        img = img_list[i].astype("float32")
        ax = plt.subplot(1, n, i+1)
        ax.set_title(titles[i])
        plt.axis("off")
        plt.imshow(img)
    plt.show()

In [ ]:
titles = ["original", "bilateral", "nl_means", "tv_chambolle"]
images, _ = next(X_test.as_numpy_iterator())
img = images[0]
img_list = [img]
img_list.append(denoise_bilateral(img, multichannel=True))
img_list.append(denoise_nl_means(img, multichannel=True))
img_list.append(denoise_tv_chambolle(img, multichannel=True))
show_images(img_list, titles)

In [ ]:
y_pred = get_predictions(X_test)
print_results(y_test, y_pred)

In [ ]:
pred_list = denoise_and_predict(X_test, y_test, denoise="bilateral")
print_results(y_test, pred_list)

In [ ]:
pred_list = denoise_and_predict(X_test, y_test, denoise="nl_means")
print_results(y_test, pred_list)

In [ ]:
pred_list = denoise_and_predict(X_test, y_test, denoise="tv_chambolle")
print_results(y_test, pred_list)

## Test with adversarial training

In [ ]:
model = keras.models.load_model("/kaggle/working/adv-traffic-sign-net")
# model = keras.models.load_model("/kaggle/input/trafficsignnet/traffic-sign-net")

In [ ]:
y_pred = get_predictions(X_test)
print_results(y_test, y_pred)

In [ ]:
pred_list = denoise_and_predict(X_test, y_test, denoise="bilateral")
print_results(y_test, pred_list)

In [ ]:
pred_list = denoise_and_predict(X_test, y_test, denoise="nl_means")
print_results(y_test, pred_list)

In [ ]:
pred_list = denoise_and_predict(X_test, y_test, denoise="tv_chambolle")
print_results(y_test, pred_list)